In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
from IPython.display import display
import IPython
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import sklearn as skl
%matplotlib notebook
%matplotlib inline
import tensorflow as tf
!pip install mglearn
import mglearn as mglearn
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

     |████████████████████████████████| 542kB 2.7MB/s 
  Created wheel for mglearn: filename=mglearn-0.1.9-py2.py3-none-any.whl size=582638 sha256=94fdd39915e0ae1c1355934fd1f3136c0efb8cfe792223f08ebadbdd270e3e83
  Stored in directory: /root/.cache/pip/wheels/eb/a6/ea/a6a3716233fa62fc561259b5cb1e28f79e9ff3592c0adac5f0
Successfully built mglearn


In [5]:
import os
os.chdir("/gdrive/My Drive/Colab Notebooks/Bottom Deeplearning/deep-learning-from-scratch-master") #디렉토리 지정하기 거기서 파일 가져오는 거고
from common.functions import *
from common.gradient import numerical_gradient

In [7]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2) # dy
        
        # backward #backpropagation 업데이트 과정 #이걸 이렇게 일일이 해야하나
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads


In [10]:
net= TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

x=np.random.rand(100,784)
y=net.predict(x)
t=np.random.rand(100,10)

grads=net.numerical_gradient(x,t)

In [13]:
print("x:\n",x)
print("y:\n",y)
print("t:\n",t)

x:
 [[0.90459625 0.80881234 0.97009285 ... 0.22153473 0.83107238 0.88928356]
 [0.03969442 0.13405264 0.70752767 ... 0.48487714 0.14636845 0.89605963]
 [0.4278765  0.21281869 0.94926902 ... 0.07654267 0.03861126 0.00384393]
 ...
 [0.09256464 0.92375306 0.33530361 ... 0.64266604 0.37073919 0.95365162]
 [0.0510801  0.79218541 0.65213741 ... 0.01083576 0.12408482 0.40635935]
 [0.78825246 0.26432159 0.18629189 ... 0.81718033 0.92254466 0.24970717]]
y:
 [[0.10255166 0.09825563 0.10069781 0.10200166 0.09834848 0.09971021
  0.09726702 0.10619522 0.10188819 0.09308411]
 [0.10261452 0.09811587 0.10067576 0.10186077 0.09857454 0.0995498
  0.09744825 0.10633466 0.10178418 0.09304165]
 [0.10287106 0.09778309 0.1005934  0.10194789 0.09872204 0.09953634
  0.09718595 0.10639637 0.10193381 0.09303005]
 [0.10255392 0.09822031 0.10048605 0.10189467 0.09836169 0.09972865
  0.09738308 0.10607805 0.10196259 0.09333098]
 [0.10255645 0.09831158 0.10053462 0.10171225 0.09858195 0.09974167
  0.09721906 0.106297

In [14]:
grads=net.numerical_gradient(x,t)